In [2]:
## Make a pickle file with the df for the given:
# exp (e.g. 'FR1')
# subs
# selected_period (e.g. 'encoding')

# this will be used to run createEventsForDF.py in parallel by Slurm

In [3]:
import pandas as pd; pd.set_option('display.max_columns', 30); pd.set_option('display.max_rows', 100)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter, ResampleFilter, MorletWaveletFilter
import xarray as xarray
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
from scipy.stats import zscore
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                         MFG_labels, IFG_labels, nonHPC_MTL_labels, ENTPHC_labels, AMY_labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *  

df = get_data_index("r1") # all RAM subjects

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [4]:
### PARAMS ###

exp = 'catFR1' # 'FR1' 'catFR1' 'RepFR1'

selected_period = 'surrounding_recall' # 'surrounding_recall' 'encoding'

### END PARAMS ###

In [5]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
elif exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
elif exp == 'RepFR1':
    exp_df = exp_df[
                    (df.subject!='R1564J') # clearly something wrong with these EEG when looking at ripple raster
                    ]
exp_df.shape
# exp_df = exp_df[257:] # for catFR1 this is R1385E-onwwards
# exp_df = exp_df[472:] # for FR1 this is R1385E-onwwards
# exp_df = exp_df[468:] # for catFR1 this is R1525J-onwards
# exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(611, 17)

In [6]:
unique_combinations = exp_df[['subject', 'session']].drop_duplicates()
num_unique_combinations = unique_combinations.shape[0]
print(f'The unique number of subject-session combinations is {num_unique_combinations}')
print(f"from {len(np.unique(exp_df.subject))} patients")

The unique number of subject-session combinations is 611
from 268 patients


In [30]:
# sub lists I've been using to explore FR1 

# subs = ['R1010J'] #'R1385E'] # ['R1065J'] #
# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus

# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2 # R1108J beatiful catFR1 data; R1065J magical subject (like with FR1)
# subs = ['R1112M','R1163T'] # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1368T','R1461T','R1463E'] # subs with particularly low localization.pair matches in pairs...see Excel in loading info on Box for details
# subs = ['R1433E','R1355T','R1367D','R1368T'] # subs with "ca3" or "dg"
# subs = ['R1001P'] # subject with over 8000 FR1 trials X electrodes
# bad_subs = ['R1433E','R1051J'] # no electrode cats
# subs = ['R1379E','R1385E','R1387E','R1394E','R1402E'] # partial beep subs
# subs = ['R1379E','R1402E','R1396T','R1395M','R1415T','R1389J','R1404E']  # test subs for eeg offset correction
# subs = ['R1054J','R1345D','R1048E','R1328E','R1308T', # first 2 are sr ≥ 1000. 3rd is 500 Hz.
#         'R1137E','R1136N','R1094T','R1122E','R1385E']#, # nice example FR1 subs used in Fig. 2
#         'R1405E','R1486J','R1501J'] # adding in 3 catFR only patients that will go into Fig. 4
# subs = ['R1120E','R1349T','R1397D','R1332M','R1299T'] # FR1 patients with electrode search ranges limited per SWR problem sessions doc on Box
# subs = ['R1269E','R1328E','R1367D','R1397D','R1405E','R1405E','R1447M','R1469D'] # catFR1 patients with electrode search changes limited
# subs = ['R1065J','R1492J','R1525J'] # killer subs
# subs = ['R1030J','R1035M'] # MFG subs
# subs = ['R1207J'] #'R1035M'] #['R1108J'] # R1065J # 'R1035M' #R1086M
# subs = ['R1111M','R1108J','R1191J','R1229M','R1247P','R1264P','R1261P','R1016M','R1065J','R1191J',
#         'R1190P','R1254E','R1337E','R1118N','R1153T','R1156D'] # FR1 patients that are getting encoding memory errors 
# subs = ['R1051J','R1118N','R1154D','R1207J','R1308T','R1311T','R1329T','R1334T','R1336T','R1334T',
#         'R1342M','R1212P','R1346T','R1311T','R1323T','R1329T','R1342M','R1346T','R1367D','R1569T'] # catFR1 memory error patients

# subs = ['R1001P','R1002P','R1003P','R1006P','R1010J','R1018P','R1020J','R1022J', # mimicking test session
#         'R1023J','R1026D','R1027J','R1030J','R1031M','R1032D','R1033D','R1034D']
# subs = ['R1367D'] #['R1525J'] #['R1367D'] #R1065J'

# # FR1 subs 2024-08-05
# subs = ['R1010J','R1054J','R1048E','R1328E','R1308T',
#         'R1137E','R1136N','R1122E','R1112M','R1108J'] # removed R1094T who used Spanish
# catFR1 subs 2024-08-05
subs = ['R1065J','R1108J','R1112M','R1122E','R1136N','R1137E',
       'R1308T','R1328E','R1405E','R1486J','R1492J','R1501J','R1525J',
       'R1485J','R1530J','R1536J','R1541T']

sub_df = exp_df[(exp_df.subject.isin(subs))  & (exp_df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[0:1]
len(sub_df)
sub_df

52

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
394,NaN,protocols/r1/subjects/R1065J/experiments/catFR...,protocols/r1/subjects/R1065J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1065J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1065J/localizations/0/m...,NaN,0,R1065J,R1065J,NaN,protocols/r1/subjects/R1065J/experiments/catFR...
395,NaN,protocols/r1/subjects/R1065J/experiments/catFR...,protocols/r1/subjects/R1065J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1065J/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1065J/localizations/0/m...,NaN,1,R1065J,R1065J,NaN,protocols/r1/subjects/R1065J/experiments/catFR...
396,NaN,protocols/r1/subjects/R1065J/experiments/catFR...,protocols/r1/subjects/R1065J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1065J/experiments/catFR...,0,NaN,2,protocols/r1/subjects/R1065J/localizations/0/m...,NaN,2,R1065J,R1065J,NaN,protocols/r1/subjects/R1065J/experiments/catFR...
397,NaN,protocols/r1/subjects/R1065J/experiments/catFR...,protocols/r1/subjects/R1065J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1065J/experiments/catFR...,0,NaN,3,protocols/r1/subjects/R1065J/localizations/0/m...,NaN,3,R1065J,R1065J,NaN,protocols/r1/subjects/R1065J/experiments/catFR...
619,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,0,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
620,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,1,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
621,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,2,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,2,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
622,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,3,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,3,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
667,NaN,protocols/r1/subjects/R1112M/experiments/catFR...,protocols/r1/subjects/R1112M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1112M/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1112M/localizations/0/m...,NaN,0,R1112M,R1112M,1.0,protocols/r1/subjects/R1112M/experiments/catFR...
1718,NaN,protocols/r1/subjects/R1328E/experiments/catFR...,protocols/r1/subjects/R1328E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1328E/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1328E/localizations/0/m...,NaN,0,R1328E,R1328E,3.1,protocols/r1/subjects/R1328E/experiments/catFR...


In [21]:
np.unique(sub_df.subject)

array(['R1065J', 'R1108J', 'R1112M', 'R1328E', 'R1405E', 'R1485J',
       'R1486J', 'R1492J', 'R1501J', 'R1525J', 'R1530J', 'R1536J',
       'R1541T'], dtype=object)

# in this case just take the full df for these 10 subs

In [31]:
final_df = exp_df #sub_df

len(final_df)
final_df.head()
final_df.tail()

611

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
32,NaN,protocols/r1/subjects/R1004D/experiments/catFR...,protocols/r1/subjects/R1004D/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1004D/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1004D/localizations/0/m...,NaN,0,R1004D,R1004D,NaN,protocols/r1/subjects/R1004D/experiments/catFR...
53,NaN,protocols/r1/subjects/R1013E/experiments/catFR...,protocols/r1/subjects/R1013E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1013E/experiments/catFR...,1,NaN,0,protocols/r1/subjects/R1013E/localizations/0/m...,NaN,0,R1013E,R1013E_1,NaN,protocols/r1/subjects/R1013E/experiments/catFR...
66,NaN,protocols/r1/subjects/R1015J/experiments/catFR...,protocols/r1/subjects/R1015J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1015J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1015J/localizations/0/m...,NaN,0,R1015J,R1015J,NaN,protocols/r1/subjects/R1015J/experiments/catFR...
72,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,0,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
73,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,1,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...


,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
3084,NaN,protocols/r1/subjects/R1683A/experiments/catFR...,protocols/r1/subjects/R1683A/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1683A/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1683A/localizations/0/m...,NaN,1,R1683A,R1683A,3.4,protocols/r1/subjects/R1683A/experiments/catFR...
3085,NaN,protocols/r1/subjects/R1683A/experiments/catFR...,protocols/r1/subjects/R1683A/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1683A/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1683A/localizations/0/m...,NaN,2,R1683A,R1683A,3.4,protocols/r1/subjects/R1683A/experiments/catFR...
3086,NaN,protocols/r1/subjects/R1683A/experiments/catFR...,protocols/r1/subjects/R1683A/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1683A/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1683A/localizations/0/m...,NaN,3,R1683A,R1683A,3.4,protocols/r1/subjects/R1683A/experiments/catFR...
3115,NaN,protocols/r1/subjects/R1689J/experiments/catFR...,protocols/r1/subjects/R1689J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1689J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1689J/localizations/0/m...,NaN,0,R1689J,R1689J,4.0,protocols/r1/subjects/R1689J/experiments/catFR...
3133,NaN,protocols/r1/subjects/R1696L/experiments/catFR...,protocols/r1/subjects/R1696L/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1696L/experiments/catFR...,0,CatFR1,NaN,protocols/r1/subjects/R1696L/localizations/0/m...,NaN,0,R1696L,R1696L,3.4,protocols/r1/subjects/R1696L/experiments/catFR...


In [32]:
save_df = 1

if save_df == 1:

    import dill
    print("SAVING TEMP DF")
    with open(f'/scratch/john/SWRrefactored/patient_info/temp_dfSWR_{selected_period}_{exp}.p', 'wb') as f: 
        dill.dump(list(final_df.itertuples()) ,f)

    params = []
    for i in range(len(list(final_df.itertuples()) )):
        params.append(i)

    print("Number of sessions in final_df: ", len(params))
    print("From these subjects:")
    print(np.unique(final_df.subject))

SAVING TEMP DF
Number of sessions in final_df:  611
From these subjects:
['R1004D' 'R1013E' 'R1015J' 'R1016M' 'R1021D' 'R1024E' 'R1026D' 'R1028M'
 'R1031M' 'R1032D' 'R1035M' 'R1036M' 'R1039M' 'R1041M' 'R1042M' 'R1045E'
 'R1050M' 'R1056M' 'R1060M' 'R1061T' 'R1065J' 'R1066P' 'R1067P' 'R1069M'
 'R1074M' 'R1075J' 'R1083J' 'R1086M' 'R1089P' 'R1092J' 'R1093J' 'R1094T'
 'R1102P' 'R1105E' 'R1106M' 'R1107J' 'R1108J' 'R1111M' 'R1112M' 'R1114C'
 'R1119P' 'R1127P' 'R1130M' 'R1131M' 'R1135E' 'R1138T' 'R1141T' 'R1144E'
 'R1147P' 'R1157C' 'R1158T' 'R1163T' 'R1167M' 'R1169P' 'R1170J' 'R1171M'
 'R1174T' 'R1176M' 'R1178P' 'R1180C' 'R1181E' 'R1184M' 'R1186P' 'R1187P'
 'R1188C' 'R1189M' 'R1190P' 'R1191J' 'R1192C' 'R1201P' 'R1202M' 'R1204T'
 'R1207J' 'R1212P' 'R1215M' 'R1217T' 'R1221P' 'R1222M' 'R1226D' 'R1227T'
 'R1228M' 'R1229M' 'R1230J' 'R1231M' 'R1235E' 'R1236J' 'R1239E' 'R1240T'
 'R1243T' 'R1245E' 'R1247P' 'R1251M' 'R1254E' 'R1260D' 'R1261P' 'R1264P'
 'R1266J' 'R1269E' 'R1271P' 'R1273D' 'R1274T' 'R127

# to check out df for a specific region:

In [21]:
selected_region = ENTPHC_labels #ENTPHC_labels #

import warnings
# Ignore specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Lab-specific readers may be moved to the cmlreaders package")
warnings.filterwarnings("ignore", category=UserWarning, message="Could not load MNI coordinates")
warnings.filterwarnings("ignore", category=UserWarning, message="Lab-specific readers may be moved to the cmlreaders package")
warnings.filterwarnings("ignore", category=FutureWarning, message="Could not load MNI coordinates")
warnings.filterwarnings("ignore", category=FutureWarning, message="Lab-specific readers may be moved to the cmlreaders package")
warnings.filterwarnings("ignore", category=FutureWarning, message="pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead")

remove_soz_ictal = 0

# get region label
if selected_region == HPC_labels:
    region_name = 'HPC'
elif selected_region == ENT_labels:
    region_name = 'ENT'
elif selected_region == PHC_labels:
    region_name = 'PHC'
elif selected_region == temporal_lobe_labels:
    region_name = 'TEMPORALLOBE'
elif selected_region == MFG_labels:
    region_name = 'MFG'
elif selected_region == IFG_labels:
    region_name = 'IFG'
elif selected_region == nonHPC_MTL_labels:
    region_name = 'nonHPC_MTL'    
elif selected_region == ENTPHC_labels:
    region_name = 'ENTPHC' 
elif selected_region == AMY_labels:
    region_name = 'AMY'

data = {
    'HPC_names': [],
    'sub_names': [],
    'sub_sess_names': [],
    'electrode_labels': [],
    'channel_nums': []
}

# Initialize a set to keep track of subjects that pass the if statement
passed_subjects = set()

for row in sub_df.itertuples():   
    sub = row.subject
    session = row.session
    exp = row.experiment
    mont = int(row.montage)
    loc = int(row.localization)
    reader = CMLReadDFRow(row)
    evs = reader.load('task_events')
    word_evs = evs[evs['type'] == 'WORD']

    pairs = reader.load('pairs')
    try:
        localizations = reader.load('localization')
    except:
        localizations = []
    tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
    elec_regions, atlas_type, pair_number, has_stein_das = get_elec_regions(localizations, pairs)

    try:
        elec_cats = reader.load('electrode_categories')
    except:
        if remove_soz_ictal == True:
            print("Remove soz ictal is true")
            add_session_to_exclude_list("Remove soz ictal is true")
            sys.exit()
        else: 
            elec_cats = []
  
    bad_bp_mask = getBadChannels(tal_struct, elec_cats, remove_soz_ictal)

    electrode_search_range = getElectrodeRanges(elec_regions, exp, sub, session, mont)

    for channel in electrode_search_range:
        if (elec_regions[channel] in selected_region) & (bad_bp_mask[channel] == 0):
            data['HPC_names'].append(elec_regions[channel])
            data['sub_names'].append(sub)
            data['sub_sess_names'].append(f"{sub}-{session}")
            data['electrode_labels'].append(pairs.iloc[channel].label)
            data['channel_nums'].append(channel)
#             print('Cumulative array size after channel: ' + str(channel))
            passed_subjects.add(sub)

cliff_notes_df = pd.DataFrame(data)

# Create final_df
selected_sub_df = sub_df[sub_df['subject'].isin(passed_subjects)]
print('Done making df for region: '+region_name)
selected_sub_df

Done making df for region: ENTPHC


,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
619,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,0,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
620,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,1,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
621,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,2,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,2,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
622,NaN,protocols/r1/subjects/R1108J/experiments/catFR...,protocols/r1/subjects/R1108J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1108J/experiments/catFR...,0,NaN,3,protocols/r1/subjects/R1108J/localizations/0/m...,NaN,3,R1108J,R1108J,NaN,protocols/r1/subjects/R1108J/experiments/catFR...
667,NaN,protocols/r1/subjects/R1112M/experiments/catFR...,protocols/r1/subjects/R1112M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1112M/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1112M/localizations/0/m...,NaN,0,R1112M,R1112M,1.0,protocols/r1/subjects/R1112M/experiments/catFR...
2020,NaN,protocols/r1/subjects/R1405E/experiments/catFR...,protocols/r1/subjects/R1405E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1405E/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1405E/localizations/0/m...,NaN,0,R1405E,R1405E,3.3,protocols/r1/subjects/R1405E/experiments/catFR...
2021,NaN,protocols/r1/subjects/R1405E/experiments/catFR...,protocols/r1/subjects/R1405E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1405E/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1405E/localizations/0/m...,NaN,1,R1405E,R1405E,3.3,protocols/r1/subjects/R1405E/experiments/catFR...
2022,NaN,protocols/r1/subjects/R1405E/experiments/catFR...,protocols/r1/subjects/R1405E/localizations/1/m...,catFR1,build,1,protocols/r1/subjects/R1405E/experiments/catFR...,1,NaN,NaN,protocols/r1/subjects/R1405E/localizations/1/m...,NaN,2,R1405E,R1405E_1,3.3,protocols/r1/subjects/R1405E/experiments/catFR...
2252,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,0,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/catFR...
2253,NaN,protocols/r1/subjects/R1486J/experiments/catFR...,protocols/r1/subjects/R1486J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1486J/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1486J/localizations/0/m...,NaN,1,R1486J,R1486J,3.4,protocols/r1/subjects/R1486J/experiments/catFR...


In [23]:
print(f'Sub-sessions for {selected_region}')
np.unique(cliff_notes_df.sub_sess_names)
print('Unique subs:')
print(np.unique(cliff_notes_df.sub_names))

Sub-sessions for ['left ec', 'right ec', 'entorhinal', ' left ent entorhinal area', ' right ent entorhinal area', 'left ent entorhinal area', 'right ent entorhinal area', '"erc"', 'erc', 'left phc', 'right phc', 'parahippocampal', ' left phg parahippocampal gyrus', ' right phg parahippocampal gyrus', 'left phg parahippocampal gyrus', 'right phg parahippocampal gyrus', '"phc"', 'phc']


array(['R1108J-0', 'R1108J-1', 'R1108J-2', 'R1108J-3', 'R1112M-0',
       'R1405E-0', 'R1405E-1', 'R1405E-2', 'R1486J-0', 'R1486J-1',
       'R1486J-2', 'R1486J-3', 'R1501J-6', 'R1501J-8', 'R1525J-0',
       'R1525J-2', 'R1525J-3', 'R1525J-4', 'R1525J-5', 'R1525J-6'],
      dtype=object)

Unique subs:
['R1108J' 'R1112M' 'R1405E' 'R1486J' 'R1501J' 'R1525J']
